In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import time
import tensorflow as tf

# train
import torch
from torch import nn
from torch.nn import functional as F

# load data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),  # mean value = 0.1307, standard deviation value = 0.3081
])

print(transform)

Compose(
    ToTensor()
    Normalize(mean=(0.1307,), std=(0.3081,))
)


In [27]:
data_path = './MNIST'

training_set = datasets.MNIST(root = data_path, train= True, download=True, transform= transform)
testing_set = datasets.MNIST(root = data_path, train= False, download=True, transform= transform)

In [25]:
batch_size = 32
lr=0.5
n_epochs = 5
no_cuda = True
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [28]:
train_loader = DataLoader(dataset = training_set, batch_size=, shuffle=True)
train_loader = DataLoader(dataset = training_set, batch_size, shuffle=True)

SyntaxError: positional argument follows keyword argument (<ipython-input-28-99b8976b2e15>, line 1)

In [7]:
class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        
        # construct layers for a neural network
        self.classifier1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=20*20),
            nn.Sigmoid(),
        ) 
        self.classifier2 = nn.Sequential(
            nn.Linear(in_features=20*20, out_features=10*10),
            nn.Sigmoid(),
        ) 
        self.classifier3 = nn.Sequential(
            nn.Linear(in_features=10*10, out_features=10),
            nn.LogSoftmax(dim=1),
        ) 
        
        
    def forward(self, inputs):                 # [batchSize, 1, 28, 28]
        x = inputs.view(inputs.size(0), -1)    # [batchSize, 28*28]
        x = self.classifier1(x)                # [batchSize, 20*20]
        x = self.classifier2(x)                # [batchSize, 10*10]
        out = self.classifier3(x)              # [batchSize, 10]
        
        return out

In [9]:
criterion = nn.NLLLoss()
loss = criterion()

NLLLoss()


In [26]:

classifier = classification().to(device)
optimizer = torch.optim.SGD(classifier.parameters(), lr)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.5
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [ ]:
def train

## 1. Plot the training and testing losses with a batch size of 32 [4pt]

## 2. Plot the training and testing accuracies with a batch size of 32 [4pt]

## 3. Plot the training and testing losses with a batch size of 64 [4pt]

## 4. Plot the training and testing accuracies with a batch size of 64 [4pt]

## 5. Plot the training and testing losses with a batch size of 128 [4pt]

## 6. Plot the training and testing accuracies with a batch size of 128 [4pt]

## 7. Print the loss at convergence with different mini-batch sizes [3pt]

## 8. Print the accuracy at convergence with different mini-batch sizes [3pt]